In [1]:
# https://stackabuse.com/text-summarization-with-nltk-in-python/

import bs4 as bs  
import urllib.request  
import re
import nltk

# 주어진 url 문서 읽기 
#url = 'https://projects.voanews.com/greenland/'
url = 'https://en.wikipedia.org/wiki/Artificial_intelligence'
#url = 'https://en.wikipedia.org/wiki/Machine_learning'
scraped_data = urllib.request.urlopen(url)  
article = scraped_data.read()


In [2]:
parsed_article = bs.BeautifulSoup(article,'lxml')
paragraphs = parsed_article.find_all('p')
article_text = ""

for p in paragraphs:  
    article_text += p.text

In [3]:
article_text

'\nArtificial intelligence (AI), is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals.[3] Colloquially, the term "artificial intelligence" is often used to describe machines (or computers) that mimic "cognitive" functions that humans associate with the human mind, such as "learning" and "problem solving."[4]\nAs machines become increasingly capable, tasks considered to require "intelligence" are often removed from the definition of AI, a phenomenon known as the AI effect.[5] A quip in Tesler\'s Theorem says "AI is whatever hasn\'t been done yet."[6] For instance, optical character recognition is frequently excluded from things considered to be AI,[7] having become a routine technology.[8] Modern machine capabilities generally classified as

In [4]:
# 전처리

# 대괄호 및 대괄호로 묶인 숫자 제거
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)  
article_text = re.sub(r'\s+', ' ', article_text)  

In [5]:
# 문장 부호, 특수 문자, 숫자 제거
formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )  
formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)  

In [6]:
formatted_article_text

' Artificial intelligence AI is intelligence demonstrated by machines unlike the natural intelligence displayed by humans and animals Leading AI textbooks define the field as the study of intelligent agents any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals Colloquially the term artificial intelligence is often used to describe machines or computers that mimic cognitive functions that humans associate with the human mind such as learning and problem solving As machines become increasingly capable tasks considered to require intelligence are often removed from the definition of AI a phenomenon known as the AI effect A quip in Tesler s Theorem says AI is whatever hasn t been done yet For instance optical character recognition is frequently excluded from things considered to be AI having become a routine technology Modern machine capabilities generally classified as AI include successfully understanding human speech com

In [7]:
# text를 sentences로 변환
sentence_list = nltk.sent_tokenize(article_text)


In [8]:
len(sentence_list)

372

In [9]:
sentence_list[3]

'As machines become increasingly capable, tasks considered to require "intelligence" are often removed from the definition of AI, a phenomenon known as the AI effect.'

In [10]:
stopwords = nltk.corpus.stopwords.words('english')
word_frequencies = {}  
for word in nltk.word_tokenize(formatted_article_text):
    word = word.lower() # 상황에 따라 적용하거나 하지 않거나
    if word not in stopwords:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1

In [11]:
word_frequencies

{'artificial': 58,
 'intelligence': 77,
 'ai': 143,
 'demonstrated': 1,
 'machines': 26,
 'unlike': 7,
 'natural': 11,
 'displayed': 2,
 'humans': 28,
 'animals': 2,
 'leading': 3,
 'textbooks': 1,
 'define': 1,
 'field': 17,
 'study': 6,
 'intelligent': 18,
 'agents': 8,
 'device': 2,
 'perceives': 2,
 'environment': 6,
 'takes': 3,
 'actions': 9,
 'maximize': 4,
 'chance': 4,
 'successfully': 6,
 'achieving': 3,
 'goals': 15,
 'colloquially': 1,
 'term': 7,
 'often': 14,
 'used': 15,
 'describe': 3,
 'computers': 11,
 'mimic': 3,
 'cognitive': 5,
 'functions': 2,
 'associate': 1,
 'human': 60,
 'mind': 18,
 'learning': 29,
 'problem': 20,
 'solving': 8,
 'become': 7,
 'increasingly': 2,
 'capable': 8,
 'tasks': 5,
 'considered': 9,
 'require': 4,
 'removed': 1,
 'definition': 3,
 'phenomenon': 3,
 'known': 8,
 'effect': 3,
 'quip': 1,
 'tesler': 1,
 'theorem': 1,
 'says': 1,
 'whatever': 1,
 'done': 1,
 'yet': 1,
 'instance': 2,
 'optical': 1,
 'character': 2,
 'recognition': 7,
 'fr

In [12]:
max(word_frequencies.values())

143

In [13]:
# 가중치가 부여된 word frequency 계산 - 가장 빈도수가 높은 단어의 빈도수로 나눔

maximum_frequency = max(word_frequencies.values())
for word in word_frequencies.keys():  
    word_frequencies[word] = (word_frequencies[word]/maximum_frequency)

In [14]:
# 문장 점수 계산 ver1

sentence_scores = {}  
for sent in sentence_list:  
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]
    sentence_scores[sent]

KeyError: 'Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals.'

In [15]:
# 문장 점수 계산 ver2
"""
sentence_scores = {}  
for sent in sentence_list:  
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]
    if len(sent.split(' ')) < 30:
        wlist = [w for w in nltk.word_tokenize(sent.lower()) if w not in stopwords and re.sub('[^a-zA-Z]', '', w) == w]
        sentlength = len(set(wlist))
        sentence_scores[sent] /= sentlength
"""

"\nsentence_scores = {}  \nfor sent in sentence_list:  \n    for word in nltk.word_tokenize(sent.lower()):\n        if word in word_frequencies.keys():\n            if len(sent.split(' ')) < 30:\n                if sent not in sentence_scores.keys():\n                    sentence_scores[sent] = word_frequencies[word]\n                else:\n                    sentence_scores[sent] += word_frequencies[word]\n    if len(sent.split(' ')) < 30:\n        wlist = [w for w in nltk.word_tokenize(sent.lower()) if w not in stopwords and re.sub('[^a-zA-Z]', '', w) == w]\n        sentlength = len(set(wlist))\n        sentence_scores[sent] /= sentlength\n"

In [16]:
sentence_scores

{' Artificial intelligence (AI), is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals.': 3.5594405594405596}

In [17]:
# 문서 요약

import heapq  
summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)
summary_sentences

[' Artificial intelligence (AI), is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals.']

In [18]:
summary = ' '.join(summary_sentences)  
print(summary) 

summary_sentences

 Artificial intelligence (AI), is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals.


[' Artificial intelligence (AI), is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals.']